# [![logo](logo.png)](https://github.com/BjornFJohansson/ypkpathway#-ypkpathway) pYPK0_RPL22A_EcacpP_TDH3

Assembly of a [_Saccharomyces cerevisiae_](http://www.yeastgenome.org) transcriptional unit vector.

Made from a linear vector fragment and three PCR products:

- a linearized pYPK0 vector for maintenance in _S. cerevisiae_ or _E. coli_ (red dashed line in figure below)
- a promoter PCR product from the `pYPKa_Z_RPL22A` vector
- a gene PCR product from the `pYPKa_A_EcacpP` vector
- a terminator PCR product from the `pYPKa_E_TDH3` vector

The four linear DNA fragments are joined by _in-vivo_ homologous recombination in a
[_Saccharomyces cerevisiae_](http://wiki.yeastgenome.org/index.php/Commonly_used_strains) strain.

![tu](tu.png "tu")

A part of the [pydna](https://pypi.python.org/pypi/pydna/) package is imported in the code cell below.

In [1]:
from pydna.parsers import parse_primers
from pydna.readers import read
from pydna.amplify import pcr
from pydna.assembly import Assembly

The Yeast Pathway Kit [standard primers](standard_primers.fasta) are read into a dictionary in the code cell below.

In [2]:
p = {x.name: x for x in parse_primers("standard_primers.fasta")}

The backbone vector [pYPK0](pYPK0.gb) is read from a local file in the code cell below.

In [3]:
backbone = read("pYPK0.gb")

The backbone vector is linearized by digestion with [MfeI](https://www.google.com/search?q=MfeI).

In [4]:
from Bio.Restriction import MfeI

In [5]:
linear_backbone = backbone.linearize(MfeI)

The pYPKa derived _E. coli_ plasmids containing
- [promoter](pYPKa_Z_RPL22A.gb)
- [gene](pYPKa_A_EcacpP.gb)
- [terminator](pYPKa_E_TDH3.gb)

are read into three variables below.

In [6]:
promoter_template   = read("pYPKa_Z_RPL22A.gb")
gene_template       = read("pYPKa_A_EcacpP.gb")
terminator_template = read("pYPKa_E_TDH3.gb")

### PCR
Three DNA fragments are PCR amplified using [standard primers](standard_primers.fasta).

[Suggested PCR programs](#Suggested-PCR-programs) can be found at the end of this document.

In [7]:

fp_prom = p['577_crp585-557']
rp_prom = p['567_pCAPsAjiIF']
fp_gene = p['468_pCAPs_release_fw']
rp_gene = p['467_pCAPs_release_re']
fp_term = p['568_pCAPsAjiIR']
rp_term = p['578_crp42-70']

In [8]:
prom = pcr(fp_prom, rp_prom, promoter_template)
gene = pcr(fp_gene, rp_gene, gene_template)
term = pcr(fp_term, rp_term, terminator_template)

In [9]:
prom.name = "pYPKa_Z_RPL22A"[8:]
gene.name = "pYPKa_A_EcacpP"[8:]
term.name = "pYPKa_E_TDH3"[8:]


The fragments will be assembled by _in-vivo_ [homologous recombination](http://www.ncbi.nlm.nih.gov/pubmed/2828185):

In [10]:
asm = Assembly((linear_backbone, prom, gene, term), limit=31)
asm

Assembly
fragments..: 5766bp 658bp 326bp 1038bp
limit(bp)..: 31
G.nodes....: 10
algorithm..: common_sub_strings

The Assembly object above should normally indicate four fragments and eight nodes.

In [11]:
candidates = asm.assemble_circular()
candidates

[Contig(o7153), Contig(o7153), Contig(o7153), Contig(o6713), Contig(o6713)]

There should normally be two candidates of equal size. These sequences should be identical.

In [12]:
candidate, *rest = candidates

In [13]:
candidate.cseguid() == rest[0].cseguid()

True

In [14]:
candidate.figure()

 -|pYPK0_lin|168
|            \/
|            /\
|            168|RPL22A|48
|                       \/
|                       /\
|                       48|EcacpP|37
|                                 \/
|                                 /\
|                                 37|TDH3|276
|                                         \/
|                                         /\
|                                         276-
|                                            |
 --------------------------------------------

The candidate vector is synchronized to the 577 primer. This means that
the plasmid origin is shifted so that it matches the backbone vector.

In [15]:
result = candidate.synced(fp_prom)

### Diagnostic PCR confirmation

The structure of the final vector is confirmed by two
separate PCR reactions, one for the promoter and gene and
one for the gene and terminator.

PCR using standard primers 577 and 467 to amplify promoter and gene.

In [16]:
product = pcr( fp_prom, rp_gene, result)

A correct clone should give this size in base pairs:

In [17]:
print(len(product))

934


If the promoter is missing from the assembly, the PCR product will have this size in base pairs:

In [18]:
print(len(product) - len(prom))

276


If the gene is missing from the assembly, the PCR product will have this size in base pairs:

In [19]:
print(len(product) - len(gene))

608


PCR using standard primers 468 and 578 to amplify gene and terminator.

In [20]:
product2 = pcr( fp_gene, rp_term, result)

A correct clone should give this size:

In [21]:
print(len(product2))

1327


If the gene is missing from the assembly, the PCR product will have this size in base pairs:

In [22]:
print(len(product2) - len(gene))

1001


If the terminator is missing from the assembly, the PCR product will have this size in base pairs:

In [23]:
print(len(product2) - len(term))

289


The cseguid checksum for the resulting plasmid is calculated for future reference.
The [cseguid checksum](http://pydna.readthedocs.org/en/latest/pydna.html#pydna.utils.cseguid)
uniquely identifies a circular double stranded sequence.

In [24]:
print(result.cseguid())

YBYJG2qluycIJ-5n_OP9aO8I0So


The file name is based on the promoter, gene and terminator designations.

In [25]:
result.locus = f"tu:{gene.name}"[:16]
result.definition = "pYPK0_RPL22A_EcacpP_TDH3"

Sequence is stamped with cseguid checksum. This can be used to verify the
integrity of the sequence file.

In [26]:
result.stamp("cSEGUID")

YBYJG2qluycIJ-5n_OP9aO8I0So

Write sequence to a local file.

In [27]:
result.write(f"{result.definition}.gb")

pYPK0_RPL22A_EcacpP_TDH3.gb

### Suggested PCR programs

For the [amplification](#PCR) of promoter, gene and terminator.

In [28]:
print(prom.forward_primer.name, prom.reverse_primer.name)
print(prom.program())

577_crp585-557 567_pCAPsAjiIF
|95°C|95°C               |    |tmf:64.6
|____|_____          72°C|72°C|tmr:69.7
|3min|30s  \ 55.9°C _____|____|45s/kb
|    |      \______/ 0:30|5min|GC 35%
|    |       30s         |    |658bp


In [29]:
print(gene.forward_primer.name, gene.reverse_primer.name)
print(gene.program())

468_pCAPs_release_fw 467_pCAPs_release_re
|95°C|95°C               |    |tmf:76.9
|____|_____          72°C|72°C|tmr:67.9
|3min|30s  \ 60.0°C _____|____|45s/kb
|    |      \______/ 0:30|5min|GC 49%
|    |       30s         |    |326bp


In [30]:
print(term.forward_primer.name, term.reverse_primer.name)
print(term.program())

568_pCAPsAjiIR 578_crp42-70
|95°C|95°C               |    |tmf:66.1
|____|_____          72°C|72°C|tmr:65.0
|3min|30s  \ 57.1°C _____|____|45s/kb
|    |      \______/ 0:46|5min|GC 38%
|    |       30s         |    |1038bp
